In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import streamlit as st
import torch
import torch.nn as nn
import numpy as np
from PIL import Image
import io
import cv2

# Define the same model architecture used during training
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32, 128),
            nn.ReLU(),
            nn.Linear(128, 4)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

# Load model
model = SimpleCNN()
model.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/archive (28)/signature_cnn_model.pth', map_location=torch.device('cpu')))
model.eval()

# Streamlit UI
st.title("🖋️ Signature Bounding Box Detection")
st.write("Upload a signature image")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Load and convert image
    image = Image.open(uploaded_file).convert("RGB")
    original_image = np.array(image)

    # Preprocess
    image_resized = image.resize((128, 128))
    image_tensor = torch.tensor(np.array(image_resized), dtype=torch.float32).permute(2, 0, 1) / 255.0
    image_tensor = image_tensor.unsqueeze(0)

    # Predict
    with torch.no_grad():
        output = model(image_tensor).squeeze().numpy()

    # Denormalize bounding box
    h, w = original_image.shape[:2]
    x_center = int(output[0] * w)
    y_center = int(output[1] * h)
    box_width = int(output[2] * w)
    box_height = int(output[3] * h)

    x1 = max(x_center - box_width // 2, 0)
    y1 = max(y_center - box_height // 2, 0)
    x2 = min(x_center + box_width // 2, w)
    y2 = min(y_center + box_height // 2, h)

    # Draw box
    boxed_image = original_image.copy()
    cv2.rectangle(boxed_image, (x1, y1), (x2, y2), (0, 255, 0), 3)

    # Show image with box
    st.subheader("📦 Predicted Bounding Box")
    st.image(boxed_image, caption="Predicted Signature Region", channels="RGB")

    # Crop and show signature
    cropped_signature = original_image[y1:y2, x1:x2]
    st.subheader("🖊️ Cropped Signature Area")
    st.image(cropped_signature, channels="RGB")

    # Download button
    cropped_pil = Image.fromarray(cropped_signature)
    buf = io.BytesIO()
    cropped_pil.save(buf, format="PNG")
    byte_im = buf.getvalue()
    st.download_button("📥 Download Cropped Signature", byte_im, file_name="signature_crop.png", mime="image/png")


2025-07-21 22:07:34.834 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-21 22:07:34.836 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-21 22:07:34.837 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-21 22:07:34.839 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-21 22:07:34.840 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-21 22:07:34.841 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-21 22:07:34.843 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-21 22:07:34.844 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [5]:
!ngrok config add-authtoken 30C0SY9QoRe1sa4uU2wCkHvIQI5_2dqXNzuLiCqpESXDpccPv

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.2 MB/s eta 0:00:00


In [4]:
!pip install pyngrok

In [14]:
from pyngrok import ngrok
ngrok.kill()  # This will close all open ngrok tunnels


In [7]:
!ls -l app.py

-rw-r--r-- 1 root root 1992 Jul 21 21:52 app.py


In [15]:
from pyngrok import ngrok
import time

# Optional: Re-authenticate ngrok if not already done
!ngrok config add-authtoken 30C0SY9QoRe1sa4uU2wCkHvIQI5_2dqXNzuLiCqpESXDpccPv

# Start Streamlit and wait
!streamlit run app.py &>/content/logs.txt &
import time
time.sleep(5)

# Start ngrok tunnel
from pyngrok import ngrok
public_url = ngrok.connect(addr="8501", proto="http")
print("Streamlit URL:", public_url)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Streamlit URL: NgrokTunnel: "https://b889fbb4afe6.ngrok-free.app" -> "http://localhost:8501"


In [24]:
!cat /content/logs.txt

/bin/bash: line 1: streamlit: command not found
